In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Load data from Excel
df = pd.read_excel("./Files/Student Schedule Dataset with Dimension Description.xlsx", sheet_name="Sheet1")

# Remove spaces and convert to snake_case-like style
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# 1. Students Table
df_students = df[['student_id', 'student_name', 'advisor_name']].drop_duplicates().reset_index(drop=True)

# 2. Courses Table
df_courses = df[['course_code', 'course_name', 'credits', 'instructor_name', 'days',
                 'time', 'building', 'room_number']].drop_duplicates().reset_index(drop=True)

# 3. Enrollments Table
df_enrollments = df[['student_id', 'course_code', 'term']].drop_duplicates().reset_index(drop=True)

# Display DataFrames
print("=== Students Table ===")
print(df_students)

print("\n=== Courses Table ===")
print(df_courses)

print("\n=== Enrollments Table ===")
print(df_enrollments)

# Create SQLite engine
engine = create_engine('sqlite:///university.db', echo=False)

# Save cleaned original df (if needed)
#df.to_sql('students_details', con=engine, if_exists='replace', index=False)

print("DataFrame successfully stored in the university.db SQLite database with cleaned column names.")


# Save to database with clean table names and column names
# df_students.to_sql('students_details', con=engine, if_exists='replace', index=False)
# df_courses.to_sql('courses', con=engine, if_exists='replace', index=False)
# df_enrollments.to_sql('enrollments', con=engine, if_exists='replace', index=False)

df.to_sql('students_details', con=engine, if_exists='replace', index=False)

print("DataFrames successfully stored in the university.db SQLite database with clean column names.")


=== Students Table ===
   student_id student_name advisor_name
0       SID01   Student 01    Advisor 2
1       SID02   Student 02    Advisor 3
2       SID03   Student 03    Advisor 4
3       SID04   Student 04    Advisor 5
4       SID05   Student 05    Advisor 1
5       SID06   Student 06    Advisor 2
6       SID07   Student 07    Advisor 3
7       SID08   Student 08    Advisor 4
8       SID09   Student 09    Advisor 5
9       SID10   Student 10    Advisor 1
10      SID11   Student 11    Advisor 2
11      SID12   Student 12    Advisor 3
12      SID13   Student 13    Advisor 4
13      SID14   Student 14    Advisor 5
14      SID15   Student 15    Advisor 1
15      SID16   Student 16    Advisor 2
16      SID17   Student 17    Advisor 3

=== Courses Table ===
   course_code           course_name  credits instructor_name days      time  \
0       CSC101           Intro to CS        3     Professor 1  MWF  08:00:00   
1       CSC102       Data Structures        4     Professor 2   TR  10:00:

In [ ]:
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.tools import tool
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType
from langchain import hub
from langchain_ollama import ChatOllama

import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite:///university.db', echo=False)

db = SQLDatabase.from_uri('sqlite:///university.db',sample_rows_in_table_info =3)

schema = db.table_info
llm_query_generator = ChatOllama(model="llama3.2",temperature=0.5)

def generate_sql_query(schema,question,current_query = None,error=None):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
            "system",
            f"""You are an expert assistant that creates simple SQL queries.
              Check the schema {schema} again and Fix the following errors {error} if any from {current_query} based on sqllite syntax.
            If the question is not related to the schema return select Null. Only return sql query or select NUll as query. DO NOT GIVE ANY OTHER OUTPUT.
            
            """,
        ),
        ("human", "{input}"),
        ]
    )

    chain = prompt | llm_query_generator
    ai_msg = chain.invoke(
        {
            "input": f"{question}",
        }
    )

    return ai_msg.content


question = "I am student 1. who are the students taking Professor 1's courses?"
retries = 5
error = None
query = None
while retries>0:
    print("=================================")
    query = generate_sql_query(schema,question,query,error)
    print("Query: ")
    print(query)
    print("=================================")
    try:
        result = db._execute(query)
        print("=================================")
        print(result)
        break
    except Exception as e:
        retries -=1
        print("=================================")
        print("Error: ")
        print(str(e))
        print("=================================")
        error = e


Query: 
SELECT student_name FROM students_details WHERE instructor_name = 'Professor 1'
[{'student_name': 'Student 01'}, {'student_name': 'Student 03'}, {'student_name': 'Student 05'}, {'student_name': 'Student 08'}, {'student_name': 'Student 10'}, {'student_name': 'Student 13'}, {'student_name': 'Student 15'}, {'student_name': 'Student 17'}]


In [ ]:
llm_with_tools = ChatOllama(model="llama3.2",temperature=0)

prompt = ChatPromptTemplate.from_messages(
        [
            (
            "system",
            f"You are a helpful assistant that has a tool to generate and execute sql queries and answer the questions in a simple way.",
        ),
        ("human", "{input}"),
        ]
    )

@tool
def execute_sql_query(question: str) -> str:
    """
        Tool to execute sql queries based on the query provided
    """

    query = generate_sql_query(schema,question)
    result = db._execute(query)
    return result


In [2]:
#!/usr/bin/env python3
# sql_qa.py

import sys
from sqlalchemy import create_engine, text
import pandas as pd

from langchain_community.utilities.sql_database import SQLDatabase
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate


def generate_sql_query(schema, question, llm, history, error=None):
    """
    Use an LLM to generate (or fix) a single SQL query against the given schema,
    taking into account the history of previous Q&A if necessary.
    """
    
    prev_ctx = ""
    if history:
        prev_q, prev_sql = history[-1]
        prev_ctx = (
            f"Previous question: {prev_q}\n"
            f"Previous SQL: {prev_sql}\n\n"
        )
    err_ctx = f"Last error: {error}\n\n" if error else ""
    system_message = f"""
        You are an expert assistant that writes simple SQLite queries.
        Here is the schema:
        {schema}

        {prev_ctx}{err_ctx}
        If the question is unrelated to the schema, return exactly:

            SELECT NULL;

        Respond *only* with the SQL statement (no explanation). Also provide only distinct values.
        """.strip()

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("human", "{input}")
    ])

    chain = prompt | llm
    ai_msg = chain.invoke({"input": question})
    return ai_msg.content.strip()

def format_and_print(rows, columns):
    """
    Nicely display query results.
    """
    if not rows:
        print("No results found.")
    else:
        df = pd.DataFrame(rows, columns=columns)
        print("\n" + df.to_string(index=False))

def main(db_uri,question,chat_history):

    model_name="llama3.2"
    engine = create_engine(db_uri, echo=False)
    db      = SQLDatabase.from_uri(db_uri, sample_rows_in_table_info=3)
    schema  = db.table_info  

    llm = ChatOllama(model=model_name, temperature=0.9)

    print(f" Connected to database: {db_uri}")
   
    retries    = 5
    last_error = None
    sql        = None

    while retries:
        sql = generate_sql_query(schema, question, llm, chat_history, last_error)
        print("\nGenerated SQL:")
        print(sql)
        try:
            with engine.connect() as conn:
                result = conn.execute(text(sql))
                rows    = result.fetchall()
                cols    = result.keys()

            chat_history.append((question, sql))
            format_and_print(rows, cols)
            break
        except Exception as e:
            last_error = str(e)
            print(f"\nQuery failed: {last_error}")
            retries -= 1
            if retries:
                print(f"   Retrying ({retries} attempts left)…\n")
            else:
                print("   Giving up after multiple retries.\n")
    return 

In [ ]:
from typing import Annotated, Sequence, TypedDict, Union

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# ─── 1) Define our shared “state” type ────────────────────────────────
class AgentState(TypedDict):
    # these two are required by LangGraph’s message-tracking,
    # but we won’t actually use them in this example
    messages: Annotated[Sequence[BaseMessage], add_messages]
    chat_history_messages: Annotated[Union[HumanMessage, AIMessage], add_messages]

    # our own variables
    a: int
    b: int
    result: int

# ─── 2) Define tools that take the whole state, not raw ints ──────────
@tool
def add_tool(state: AgentState) -> AgentState:
    """
    Pops `a` and `b` out of the state, adds them, writes
    the sum back into state['result'], and returns the state.
    """
    state['result'] = state['a'] + state['b']
    return state

@tool
def subtract_tool(state: AgentState) -> AgentState:
    """
    Pops `a` and `b` out of the state, subtracts them, writes
    the difference back into state['result'], and returns the state.
    """
    state['result'] = state['a'] - state['b']
    return state

# ─── 3) Wire up the graph ─────────────────────────────────────────────
llm = ChatOllama(model="llama3.2", temperature=0)  # temperature 0 since no real generation

graph = StateGraph(AgentState)
tools = [subtract_tool,add_tool]


# ─── 4) Run it ────────────────────────────────────────────────────────
if __name__ == "__main__":
    init_state = {
        "messages": [],
        "chat_history_messages": [],
        "a": 10,
        "b": 4,
        "result": 0,
    }

    final_state = graph.run(init_state, llm=llm)
    print("Final result in state:", final_state["result"])


AttributeError: type object 'ToolNode' has no attribute 'from_tool'